In [1]:
import gensim
import spacy
import numpy as np
from scipy.stats import norm
from wefe.word_embedding_model import WordEmbeddingModel
import matplotlib.pyplot as plt

In [2]:
def cosine_similarity(v1: np.ndarray, v2: np.ndarray) -> float:
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

def compute_association(w: np.ndarray, A: np.ndarray) -> float:
    return np.mean([cosine_similarity(w, a) for a in A])

def compute_attribute_association(X: np.ndarray, A: np.ndarray) -> np.ndarray:
    return np.array([compute_association(x, A) for x in X])

def compute_attribute_association_L2(A: np.ndarray, T: np.ndarray) -> np.ndarray:
    return np.array([compute_association(a, T) for a in A])

def compute_joint_std(X_Associations: np.ndarray, Y_Associations: np.ndarray) -> float:
    return np.std(np.concatenate([X_Associations, Y_Associations]), ddof=1)

def compute_p_value(X_Diff: np.ndarray, Y_Diff: np.ndarray, permutations: int=1000) -> float:
    test_statistic = np.sum(X_Diff) - np.sum(Y_Diff)
    empirical_distribution = np.array([np.random.choice(np.concatenate([X_Diff, Y_Diff]), size=len(X_Diff) + len(Y_Diff), replace=False) for _ in range(permutations)])
    empirical_differences = np.sum(empirical_distribution[:, :len(X_Diff)], axis=1) - np.sum(empirical_distribution[:, len(X_Diff):], axis=1)
    
    return 1-norm.cdf(test_statistic, loc=np.mean(empirical_differences), scale=np.std(empirical_differences, ddof=1))

def level_1(X: np.ndarray, Y: np.ndarray, A: np.ndarray, B: np.ndarray, permutations: int=1000) -> float:
    X_Associations_A = compute_attribute_association(X, A)
    X_Associations_B = compute_attribute_association(X, B)
    X_Differential_Associations = X_Associations_A - X_Associations_B

    Y_Associations_A = compute_attribute_association(Y, A)
    Y_Associations_B = compute_attribute_association(Y, B)
    Y_Differential_Associations = Y_Associations_A - Y_Associations_B

    X_Mean = np.mean(X_Differential_Associations)
    Y_Mean = np.mean(Y_Differential_Associations)

    p_value = compute_p_value(X_Differential_Associations, Y_Differential_Associations, permutations=permutations)

    return (X_Mean - Y_Mean) / compute_joint_std(X_Differential_Associations, Y_Differential_Associations), p_value

def level_2(T: np.ndarray, A: np.ndarray, B: np.ndarray, permutations: int=1000) -> float:
    A_Associations_T = compute_attribute_association_L2(A, T)
    B_Associations_T = compute_attribute_association_L2(B, T)

    p_value = compute_p_value(A_Associations_T, B_Associations_T, permutations=permutations)

    return (np.mean(A_Associations_T) - np.mean(B_Associations_T)) / compute_joint_std(A_Associations_T, B_Associations_T), p_value

def level_3(T: np.ndarray, A: np.ndarray) -> float:
    T_Associations_A = [cosine_similarity(t, a) for t in T for a in A]
    return np.mean(T_Associations_A), np.std(T_Associations_A, ddof=1)

def ML_EAT(A: np.ndarray, B: np.ndarray, X: np.ndarray, Y: np.ndarray, permutations: int=1000) -> dict:
    L1_effect_size, L1_p_value = level_1(X, Y, A, B, permutations=permutations)
    L2_effect_size_X, L2_p_value_X = level_2(X, A, B, permutations=permutations)
    L2_effect_size_Y, L2_p_value_Y = level_2(Y, A, B, permutations=permutations)
    L3_mean_AX, L3_std_AX = level_3(X, A)
    L3_mean_BX, L3_std_BX = level_3(X, B)
    L3_mean_AY, L3_std_AY = level_3(Y, A)
    L3_mean_BY, L3_std_BY = level_3(Y, B)

    return {
        'L1_effect_size': L1_effect_size,
        'L1_p_value': L1_p_value,
        'L2_effect_size_X': L2_effect_size_X,
        'L2_p_value_X': L2_p_value_X,
        'L2_effect_size_Y': L2_effect_size_Y,
        'L2_p_value_Y': L2_p_value_Y,
        'L3_mean_AX': L3_mean_AX,
        'L3_std_AX': L3_std_AX,
        'L3_mean_BX': L3_mean_BX,
        'L3_std_BX': L3_std_BX,
        'L3_mean_AY': L3_mean_AY,
        'L3_std_AY': L3_std_AY,
        'L3_mean_BY': L3_mean_BY,
        'L3_std_BY': L3_std_BY,
    }
def get_np_embeddings(target_words: list,
                      vocab_dict: dict,
                      embeddings: np.ndarray) -> np.ndarray:
    """
    Get the embeddings for the target words.
    """

    return np.array([embeddings[vocab_dict[word]] for word in target_words])

In [3]:
# Load pre-trained Dutch FastText embeddings with Gensim
print("Loading word embeddings...")
nl_embeddings = gensim.models.KeyedVectors.load_word2vec_format("/Users/matthijstentije/University/MSc_Data-Science/Thesis/MSc_Data_Science_Thesis/data/cc.nl.300.vec.gz", binary=False)
print("Embeddings loaded.")


Loading word embeddings...
Embeddings loaded.


In [4]:
# Convert to WEFE-compatible format
model = WordEmbeddingModel(nl_embeddings, "Dutch FastText")

In [5]:
# Load spaCy Dutch NLP model
nlp = spacy.load('nl_core_news_lg')

## Extract Dutch Adjectives
Now we extract all the lemmas of adjectives in the Dutch language. Therefore, we load in the data downloaded from the Algemeen Nederlands Woordenboek (ANW), a reliable source for Dutch linguistic information.

In [6]:
import pandas as pd
import re

def get_clean_adjectives_from_tsv(file_path, embedding_model, nlp_model):
    """
    Leest een TSV-bestand in en extraheert en filtert de lemmas op basis van criteria:
    - Rijen waarbij de derde kolom exact gelijk is aan "AA(degree=pos)".
    - Schoonmaken van de waarden in de tweede kolom (lemma).
    - Filtert op woorden met meer dan 1 karakter, verwijdert duplicaten, 
      verwijdert woorden die niet in het embeddingmodel zitten en filtert vervolgens
      op Nederlandse adjectieven via spaCy.

    Parameters:
    - file_path: pad naar het TSV-bestand.
    - embedding_model: embeddingmodel waarin gecontroleerd wordt of een woord aanwezig is.
    - nlp_model: spaCy-model voor POS-tagging (bijv. "nl_core_news_sm").

    Returns:
    - Een lijst met gefilterde adjectieven.
    """
    def clean_lemma(lemma):
        """
        Zet het lemma om naar lowercase en behoudt alleen de opeenvolgende letters
        aan het begin van de string. Als het lemma niet met een letter begint, retourneert
        de functie een lege string.
        """
        lemma = lemma.lower()
        match = re.match(r"[a-z]+", lemma)
        return match.group(0) if match else ''

    # Lees het TSV-bestand in zonder header.
    df = pd.read_csv(file_path, delimiter='\t', header=None)
    
    # Filter de rijen waarbij de derde kolom exact "AA(degree=pos)" is.
    df = df[df[5] == "AA(degree=pos,case=norm,infl=e)"]
    
    # Pas de cleaning-functie toe op de tweede kolom (index 1)
    df['cleaned_lemma'] = df[1].apply(clean_lemma)
    
    # Zet de verwerkte kolom om in een lijst en verwijder lege strings.
    lemma_list = df['cleaned_lemma'].tolist()
    lemma_list = [lemma for lemma in lemma_list if lemma != '']
    
    # Filter op lemma's met meer dan 1 karakter.
    filtered_lemmas = [lemma for lemma in lemma_list if len(lemma) > 1]
    
    # Verwijder duplicaten en behoud de volgorde.
    unique_lemmas = list(dict.fromkeys(filtered_lemmas))
    print(f"Total unique lemmas: {len(unique_lemmas)}")
    
    # Identificeer woorden die niet in het embeddingmodel zitten.
    missing_words = [word for word in unique_lemmas if word not in embedding_model]
    print(f"Total missing words: {len(missing_words)}")
    print("Sample missing words:", missing_words[:10])
    
    # Houd alleen de woorden die in het embeddingmodel zitten.
    filtered_lemmas = [word for word in unique_lemmas if word in embedding_model]

    def filter_adjectives(lemmas, nlp):
        """
        Filtert de lijst met lemmas op Nederlandse adjectieven met behulp van spaCy.
        """
        adj_set = set()
        for lemma in lemmas:
            doc = nlp(lemma)
            if doc and doc[0].pos_ == 'ADJ':
                adj_set.add(lemma)
        return list(adj_set)
    
    # Filter op adjectieven via spaCy.
    filtered_adjectives = filter_adjectives(filtered_lemmas, nlp_model)
    
    # Filter woorden die specifieke doelwoorden bevatten.
    target_words = [
        'man','mannen','jongen','kerel','vader','zoon',
        'vrouw','vrouwelijk','vrouwen','meisje','dame'
    ]
    filtered_adjectives = [
        adj for adj in filtered_adjectives
        if not any(target_word in adj for target_word in target_words)
    ]

    print(f"Remaining adjectives after final filtering: {len(filtered_adjectives)}")
    
    return filtered_adjectives

file_path = '/Users/matthijstentije/Downloads/GiGaNT-Molex2.0/Data/molex_22_02_2022 2.tsv/molex_22_02_2022.tsv'
adjectives = get_clean_adjectives_from_tsv(file_path, model, nlp)
print("Gefilterde adjectieven:", adjectives)

Total unique lemmas: 16253
Total missing words: 4025
Sample missing words: ['claustrofoob', 'poepchic', 'veelgesmaad', 'aalglad', 'ijsglad', 'gelubd', 'slechtgekleed', 'zwartgekleed', 'europabreed', 'natiebreed']
Remaining adjectives after final filtering: 6439
Gefilterde adjectieven: ['schelpvormig', 'symfonisch', 'kinderachtig', 'ongestoord', 'artificieel', 'paraat', 'democratisch', 'overmoedig', 'litterair', 'inadequaat', 'wezenlijk', 'concurrentieel', 'serologisch', 'roostertechnisch', 'lofwaardig', 'middernachtelijk', 'gedesillusioneerd', 'onwaarschijnlijk', 'proportioneel', 'werkelijk', 'geestig', 'moerassig', 'transformatief', 'schemerig', 'vlijtig', 'letterlijk', 'saffraankleurig', 'herbruikbaar', 'thermodynamisch', 'congenitaal', 'overgelukkig', 'hapklaar', 'arm', 'molair', 'eufemistisch', 'hoogstwaarschijnlijk', 'arbeidsloos', 'veelvuldig', 'legaal', 'klaarblijkelijk', 'schriel', 'dom', 'pokdalig', 'borstelig', 'socialistisch', 'brandwerend', 'internationalistisch', 'psychoti

In [18]:
len(adjectives)

6439

In [33]:
def format_first_100_words(word_list):
    first_100 = word_list[6000:6500]  # Neem de eerste 100 woorden
    return "|".join(first_100)

# Voorbeeldgebruik

formatted_output = format_first_100_words(adjectives)

print(formatted_output)



etiologisch|edel|oorverdovend|onthoornd|antiallergisch|statisch|grotesk|multimediaal|eigenzinnig|wereldschokkend|ongebruikt|spijtig|krachteloos|prikkelbaar|schichtig|miniem|menselijk|energieneutraal|witgekalkt|autoritair|tweezijdig|buitengewoon|onislamitisch|onverzwakt|psychopathisch|ceremonieel|welbewust|bangelijk|rijk|terecht|oftalmologisch|drempelloos|schoon|stilistisch|nijpend|supergemotiveerd|bedrijfsklaar|kitscherig|steevast|interlineair|dagelijks|openbaar|haarzuiver|doorzoekbaar|misbaar|onvruchtbaar|verbaal|kruidig|coronair|afkeuringswaardig|bewusteloos|noordwaarts|ontredderd|opzwepend|fijnkorrelig|eenmotorig|krijtachtig|transactioneel|overbezet|precieus|kwantummechanisch|additioneel|onbeschroomd|vorderbaar|inklapbaar|hoorndol|werkloos|postmodern|zijlings|bovenstandig|vraatzuchtig|generisch|huiselijk|bio|prijstechnisch|trimestrieel|zweterig|rotsachtig|billijk|besteedbaar|burgerrechtelijk|razend|kafka|onafzienbaar|extrawettelijk|schurkachtig|auteursrechtelijk|schaapachtig|blikker

In [ ]:
def compute_individual_bias(adjectives, male_terms, female_terms, model):
    """
    Voor elk bijvoeglijk naamwoord wordt het verschil berekend tussen de gemiddelde cosine similarity
    met mannelijke termen en met vrouwelijke termen. Woorden waarin een van de target-woorden voorkomt 
    (bijvoorbeeld 'dame' in 'damesachtig') worden eerst uitgesloten.
    """
    # Combineer de target-woorden en filter de adjectives
    target_words = male_terms + female_terms
    filtered_adjectives = [adj for adj in adjectives if not any(target in adj for target in target_words)]
    
    bias_results = {}
    for adj in filtered_adjectives:
        # Controleer of de embedding voor het bijvoeglijk naamwoord beschikbaar is
        if model[adj] is None:
            continue
        
        male_sims = []
        for term in male_terms:
            vec_term = model[term]
            if vec_term is None:
                continue
            male_sims.append(cosine_similarity(model[adj], vec_term))
        
        female_sims = []
        for term in female_terms:
            vec_term = model[term]
            if vec_term is None:
                continue
            female_sims.append(cosine_similarity(model[adj], vec_term))
        
        # Bereken de bias alleen als er zowel mannelijke als vrouwelijke vergelijkingen beschikbaar zijn
        if male_sims and female_sims:
            bias_value = np.mean(male_sims) - np.mean(female_sims)
            bias_results[adj] = bias_value
    return bias_results

# Voorbeeldlijsten target woorden
target_words_man = ['man','mannen','jongen','kerel','vader','zoon','vent']
target_words_vrouw = ['vrouw','vrouwelijk','vrouwen','meisje','dame',]

# Aannemende dat `unique_lemmas` je lijst met schoongemaakte lemma's i 
# en `model` jouw word embedding model (bijv. een WEFE-wrapped FastText-model)
individual_bias = compute_individual_bias(adjectives=adjectives, male_terms=target_words_man, female_terms=target_words_vrouw, model=model)

# Sorteer de bias-resultaten op bias-waarde (van laag naar hoog)
sorted_bias = sorted(individual_bias.items(), key=lambda x: x[1])

# Selecteer de 30 laagste scores
lowest_30 = sorted_bias[:30]

# Selecteer de 30 hoogste scores
highest_30 = sorted_bias[-30:]

print("30 Laagste Scores:")
for word, score in lowest_30:
    print(f"{word}: {score:.3f}")

print("\n30 Hoogste Scores:")
for word, score in reversed(highest_30):
    print(f"{word}: {score:.3f}")



30 Laagste Scores:
sensueel: -0.202
genitaal: -0.173
feministisch: -0.168
erotisch: -0.162
genderneutraal: -0.161
huidkleurig: -0.158
modisch: -0.151
feminien: -0.151
bloemig: -0.149
aziatisch: -0.144
glamoureus: -0.140
modebewust: -0.140
meerkleurig: -0.138
voluptueus: -0.136
chic: -0.135
flirterig: -0.134
gynaecologisch: -0.134
tuttig: -0.133
elegant: -0.133
satijnachtig: -0.133
modieus: -0.130
zwoel: -0.128
seksistisch: -0.123
sierlijk: -0.123
geslachtsloos: -0.123
bekoorlijk: -0.123
zedig: -0.121
lesbisch: -0.120
beeldschoon: -0.120
klassevol: -0.119

30 Hoogste Scores:
dood: 0.101
autodidact: 0.094
apetrots: 0.092
bliksemsnel: 0.084
landwaarts: 0.082
fanatiek: 0.082
knetterhard: 0.081
balvast: 0.081
oerdom: 0.080
keihard: 0.079
doodmoe: 0.076
furieus: 0.076
ambteloos: 0.075
spijkerhard: 0.074
adoptief: 0.074
geboortig: 0.073
woensdags: 0.073
gebroederlijk: 0.072
beenhard: 0.071
speelklaar: 0.071
bergopwaarts: 0.071
eigenhandig: 0.068
zaterdags: 0.068
hartgrondig: 0.067
kameraadsch

In [13]:
# Lijst met specifieke bijvoeglijke naamwoorden waarin je geïnteresseerd bent
search_words = ["sterk", "zacht", "moedig", "emotioneel", "dominant", "zorgzaam", "aardig", "knap", "schattig"]

print("Bias scores voor specifieke woorden:")
for word in search_words:
    # Haal de bias score op, indien het woord aanwezig is in de bias resultaten
    bias_value = individual_bias.get(word)
    if bias_value is not None:
        print(f"{word}: {bias_value:.3f}")
    else:
        print(f"{word}: Niet gevonden in de bias resultaten.")


Bias scores voor specifieke woorden:
sterk: -0.015
zacht: -0.048
moedig: 0.030
emotioneel: -0.032
dominant: -0.034
zorgzaam: -0.059
aardig: -0.019
knap: -0.009
schattig: -0.077


In [9]:
from wefe.metrics import RIPA
from wefe.query import Query

# Debugging print statements
print(f"Number of adjectives (filtered_lemmas): {len(adjectives)}")
print(f"Number of Male Terms: {len(['man', 'mannelijk', 'hij', 'hem', 'kerel', 'jongen', 'mannen'])}")
print(f"Number of Female Terms: {len(['vrouw', 'vrouwelijk', 'zij', 'haar', 'dame', 'meisje', 'vrouwen'])}")

# Define the query
query = Query(
    target_sets=[
        ['man', 'mannelijk', 'hij', 'hem', 'kerel', 'jongen', 'mannen'],
        ['vrouw', 'vrouwelijk', 'zij', 'haar', 'dame', 'meisje', 'vrouwen']
    ],
    attribute_sets=[adjectives],  # Ensure it's a list of lists
    target_sets_names=["Male Terms", "Female Terms"],
    attribute_sets_names=["Adjectives"],
)

# Instantiate the metric and run the query
ripa = RIPA()
result = ripa.run_query(query, model)

# Print results
print(result)

Number of adjectives (filtered_lemmas): 6439
Number of Male Terms: 7
Number of Female Terms: 7
{'query_name': 'Male Terms and Female Terms wrt Adjectives', 'result': 0.02229223, 'ripa': 0.02229223, 'word_values': {'onschatbaar': {'mean': 0.01589501, 'std': 0.037143074}, 'nadenkend': {'mean': 0.051828604, 'std': 0.058115263}, 'maatschappijkritisch': {'mean': 0.01580679, 'std': 0.022398684}, 'onbestemd': {'mean': 0.032825876, 'std': 0.040815294}, 'transnationaal': {'mean': -0.008950884, 'std': 0.023135582}, 'eindig': {'mean': 0.020668391, 'std': 0.05911159}, 'angstig': {'mean': 0.025758808, 'std': 0.06938111}, 'zuidwaarts': {'mean': 0.024602514, 'std': 0.026485462}, 'gewestelijk': {'mean': 0.026025584, 'std': 0.035967592}, 'synoptisch': {'mean': 0.024831474, 'std': 0.031610876}, 'koppig': {'mean': 0.08949149, 'std': 0.096476845}, 'diepzinnig': {'mean': 0.025721699, 'std': 0.04310986}, 'archivalisch': {'mean': 0.018876389, 'std': 0.050606936}, 'geothermisch': {'mean': -0.0050813, 'std': 0

In [10]:
# Extract scores into a DataFrame
scores_df = pd.DataFrame({
    'Word': result['word_values'].keys(),
"Mean Score": [value['mean'] for value in result['word_values'].values()],
    "Std Dev": [value['std'] for value in result['word_values'].values()]
})

# Sort for readability
scores_df = scores_df.sort_values(by='Mean Score', ascending=False).reset_index(drop=True)
scores_df.head(n=50)

,Word,Mean Score,Std Dev
0,laf,0.247612,0.177933
1,lui,0.241594,0.219628
2,gek,0.222743,0.219595
3,zot,0.219160,0.190957
4,paf,0.207035,0.107326
5,hol,0.205209,0.112396
6,ruig,0.188516,0.142382
7,rap,0.187402,0.185352
8,tof,0.187293,0.210392
9,dik,0.184742,0.163153


In [11]:
# Extract scores into a DataFrame
scores_df = pd.DataFrame({
    'Word': result['word_values'].keys(),
"Mean Score": [value['mean'] for value in result['word_values'].values()],
    "Std Dev": [value['std'] for value in result['word_values'].values()]
})

# Sort for readability
scores_df = scores_df.sort_values(by='Mean Score', ascending=True).reset_index(drop=True)
scores_df.head(n=50)

,Word,Mean Score,Std Dev
0,duaal,-0.087122,0.067154
1,fair,-0.076598,0.122746
2,pregnant,-0.067705,0.043986
3,sensueel,-0.055795,0.086827
4,chic,-0.052720,0.138730
5,feminien,-0.050121,0.067502
6,zijdezacht,-0.048267,0.036033
7,gifvrij,-0.046918,0.026362
8,ovarieel,-0.045993,0.044732
9,genitaal,-0.044525,0.079553
